# This code is used to process the json files and extract the needed attributes to get it into the format we need

### Note: we can use the Classes instead of implementing the functions
### Gnip

In [1]:
%pylab inline
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib2
import numpy as np
import os
import os.path
from scipy import misc
from time import strftime,strptime
import re
import math
import pymysql
import MySQLdb
import os,sys
import gzip, json
from dateutil import parser

Populating the interactive namespace from numpy and matplotlib


In [2]:
def getimage(url, full_name):

    if os.path.isfile(full_name):
        return mpimg.imread(full_name)
    
    try:
        f = urllib2.urlopen(url)
    except:
        return None
    
    data = f.read()
    with open(full_name, "wb") as code:
        code.write(data)
    return mpimg.imread(full_name)


def getallimages(panda_name, column_name):
    downloaded_images = []
    for row in panda_name.loc[panda_name[column_name].notnull(),column_name]:
        for image in row:
            img = getimage(image['media_url'],image['media_url'].split('/')[-1])
            if not(img is None):
                downloaded_images.append(img)
    return downloaded_images

def klout_getId(screenname):
    url = 'http://api.klout.com/v2/identity.json/twitter?screenName={0}&key=memp3ncn4qvp6c8guzjcc8dp'.format(screenname)    
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
    
def klout_getScore(kloutId):
    url = 'http://api.klout.com/v2/user.json/{0}/score?key=memp3ncn4qvp6c8guzjcc8dp'.format(kloutId)
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
def extractImageUrl(cell):
    if cell is None:
        return None
    return cell[0]['media_url']

def convertDTToDB(cell):
    return strftime('%Y-%m-%d %H:%M:%S', strptime(cell,'%Y-%m-%dT%H:%M:%S.000Z'))
def getOldId(cell):
    return cell.split(':')[2]
def normalizeTweetText(text):
    if text is None or type(text) is float:
        return None
    return re.sub(r'[^\w#:@/\.\-\,]', ' ', text)

## Note that if we may put NULL for any of the attributes
## we should remove the single quotation marks from around them.
def ensureDBNull(cell):
    return 'NULL' if cell is None else ("'" + cell + "'")

def getTweetsInsertQuery(tweets):
    query = ""
    for tweet in tweets:
        temp = "REPLACE INTO tweet VALUES('{0}', '{1}', '{2}', '{3}', {4}, '{5}', {6}, {7}, {8});\n".format(tweet[0],
                                     tweet[1],
                                     tweet[2],
                                     tweet[3],
                                     ensureDBNull(tweet[4]),
                                     tweet[5],
                                     ensureDBNull(tweet[6]),
                                     ensureDBNull(tweet[7]),
                                     ensureDBNull(tweet[8]))
        
        query = "{0}{1}".format(query, temp)                         
    return query

def getUsersInsertQuery(users):
    query = ""
    for user in users:
        temp = "REPLACE INTO user VALUES('{0}', '{1}', {2}, {3}, \
          '{4}', '{5}', '{6}', {7}, '{8}');\n".format(user[0],
                                                   user[1],
                                                   ensureDBNull(user[2]),
                                                   user[3],
                                                   user[4],
                                                   user[5],
                                                   user[6],
                                                   ensureDBNull(user[7]),
                                                   user[8])

        query = "{0}{1}".format(query, temp)                
    return query

def getPlacesInsertQuery(places):
    query = ""
    for place in places:
        temp = "REPLACE INTO place VALUES ('{0}', '{1}', '{2}', '{3}');\n".format(place[0],
                                                     place[1],
                                                     place[2],
                                                     place[3])

        query = "{0}{1}".format(query, temp)
    return query


def convertNanToNone(text):
    try:
        if str(text) == 'nan':
            return None
    except:
        pass
    return text

def listToStr(cell):
    if cell is None:
        return None
    mystr = "";
    for i in cell:
        try:
            mystr = mystr + "," + str(i)
        except:
            print i
    mystr = "[{0}]".format(mystr[1:])
    return mystr
def getDBInstance():
    return MySQLdb.connect("uncg.saadmtsa.club",    # your host, usually localhost
                         "root",         # your username
                         "vJnVubg49U",  # your password
                         "geotwitter")
def extractHashTags(hashTags):
    if hashTags is None or hashTags == []:
        return None
    mylist = ''
    for hashtag in hashTags:
        mylist = mylist + ',' + hashtag['text']
    return mylist[1:]

In [3]:
class GnipDataProcessor(object):

    def __init__(self, i_path, collection, chunk_size=50):
        self.path = i_path
        self.chunk = []
        self.chunk_size = chunk_size
        self.collection = collection
        self.total_inserts = 0

    def all_files(self):
        for path, dirs, files in os.walk(self.path):
            for f in files:
                yield os.path.join(path, f)
    def iter_files(self):
        file_generator = self.all_files()

        for f in file_generator:
            try:
                gfile = gzip.open(f)
                for line in gfile:
                    self.process_line(line)
                gfile.close()
            except Exception as e:
                # print e
                pass
            if self.chunk != []:
                yield self.chunk
        
        

    def process_line(self, line):
        try:
            if len(self.chunk) > self.chunk_size:
                self.process_chunk()
                self.chunk = []
            if line.strip() != "":
                data = json.loads(line)
                if 'id' in data:
                    # data['postedTime_mongo'] = parser.parse(data['postedTime'])
                    self.chunk.append(line)

        except Exception as e:
            # print "error storing chunk \n"
            # print line, e.msg()
            raise

    def process_chunk(self):
        #for item in self.chunk:
        try:
            #print self.chunk
            self.total_inserts += len(self.chunk)
            # print "Inserted: %d number of docs" % self.total_inserts
        except:
            # print "issue inserting"
            pass

In [4]:
insrt = GnipDataProcessor("IRMA Data","", chunk_size=1000)
result = []
mygen = insrt.iter_files()
for i in mygen:
    result.append(i)

In [14]:
## Get the data and put it in a panda dataframe
myjson = []
# myfile = open("/home/saed/Desktop/Twitter/abc.json", 'r')
for i in result[400:500]:
    for j in i:
        try:
            myjson.append(json.loads(j))
        except:
            print j
mypanda = pd.io.json.json_normalize(myjson)

In [15]:
mydata = mypanda.copy()
mydata = mydata.drop_duplicates(subset = ['id'])
mydata = mydata[['id','postedTime','body','geo.coordinates', 'location.name',
                  'twitter_entities.media','twitter_lang', 'twitter_entities.hashtags','actor.id','actor.preferredUsername',
                  'actor.location.displayName','actor.verified','actor.followersCount','actor.friendsCount',
                  'actor.statusesCount','actor.postedTime', 'location.displayName',
                  'location.twitter_country_code','location.geo.coordinates']]
mydata = mydata.dropna(axis = 0, how = 'all')
mydata = mydata.loc[mydata['id'].notnull()]
mydata = mydata.reset_index(drop = True)
mydata = mydata.rename(index=str, columns={'id' : 'tweet.id', 'geo.coordinates' : 'tweet.coordinates',
                                  'postedTime' : 'tweet.created_at', 'body' : 'tweet.text', 'twitter_lang' : 'tweet.lang',
                                  'twitter_entities.media' : 'tweet.media', 'actor.id' : 'user.id',
                                  'actor.preferredUsername' : 'user.screen_name', 'actor.location.displayName' : 'user.location',
                                  'actor.verified' : 'user.verified', 'actor.followersCount': 'user.followers_count',
                                  'actor.friendsCount' : 'user.friends_count', 'actor.statusesCount' : 'user.statuses_count',
                                  'actor.postedTime' : 'user.created_at', 'twitter_entities.hashtags' : 'tweet.hashtags'})


mydata = mydata.applymap(convertNanToNone)
mydata['tweet.created_at'] = mydata['tweet.created_at'].apply(convertDTToDB)
mydata['user.created_at'] = mydata['user.created_at'].apply(convertDTToDB)
mydata['tweet.coordinates'] = mydata['tweet.coordinates'].apply(convertNanToNone)
mydata['tweet.coordinates'] = mydata['tweet.coordinates'].apply(listToStr)
mydata['tweet.text'] = mydata['tweet.text'].apply(normalizeTweetText)
mydata['user.location'] = mydata['user.location'].apply(normalizeTweetText)
mydata['location.name'] = mydata['location.name'].apply(normalizeTweetText)
mydata['tweet.media'] = mydata['tweet.media'].apply(extractImageUrl)
mydata['tweet.id'] = mydata['tweet.id'].apply(getOldId)
mydata['user.id'] = mydata['user.id'].apply(getOldId)
mydata['tweet.hashtags'] = mydata['tweet.hashtags'].apply(extractHashTags)
mydata['tweet.hashtags'] = mydata['tweet.hashtags'].apply(normalizeTweetText)

In [16]:
tweets = mydata[['tweet.id','tweet.created_at','tweet.text','user.id','tweet.coordinates','location.name','tweet.media','tweet.lang', 'tweet.hashtags']]
tweets

,tweet.id,tweet.created_at,tweet.text,user.id,tweet.coordinates,location.name,tweet.media,tweet.lang,tweet.hashtags
0,916979419482345472,2017-10-08 10:51:44,@birchbox I only received 4 things on my box t...,771830969934438401,None,Bayonet Point,None,en,None
1,916979495156019200,2017-10-08 10:52:02,#roadtrip #throughthewindshield #palmtrees #sk...,757948847184240646,"[27.9736,-82.7643]",Clearwater,None,und,"roadtrip,throughthewindshield,palmtrees,sky,ro..."
2,916979920915521536,2017-10-08 10:53:44,Call #TeamChais 813 928-4308 for all your #T...,24633648,"[27.7625,-82.54583333]",Florida,None,en,"TeamChais,TampaBay,RealEstate"
3,916980269848113152,2017-10-08 10:55:07,@Antiloquy How does that make any fucking sense,815777993733898240,None,Ridge Wood Heights,None,en,None
4,916980733616549889,2017-10-08 10:56:58,News Channel 8 Today is on right now @WFLAEd @...,71023584,None,"The NewsCenter TBO.com, Tampa Tribune, News C...",http://pbs.twimg.com/ext_tw_video_thumb/916980...,en,None
5,916981310165569536,2017-10-08 10:59:15,Come see our newest Bikini Barista Jessy Ju...,2284127615,"[26.96755,-82.07943]",Charlotte Harbor,None,en,relaxfulsunday
6,916981304415178753,2017-10-08 10:59:14,"Is it me u missing,just wondering",2876810176,None,Temple Terrace,None,en,None
7,916969566881943552,2017-10-08 10:12:35,I m never going to the casino again,4833325693,None,East Lake-Orient Park,None,en,None
8,916969658431033345,2017-10-08 10:12:57,And who really cares about him anyway #GoodRi...,773983063936294913,None,Tallevast,None,en,GoodRiddanceJamesWoods
9,916970214474113024,2017-10-08 10:15:10,Officially a tattoo apprentice,4833325693,None,East Lake-Orient Park,None,en,None


In [17]:
users = mydata[['user.id','user.screen_name','user.location','user.verified','user.followers_count','user.friends_count','user.statuses_count','user.created_at']]
users = users.drop_duplicates(subset = ['user.id'])
users['klout_score'] = [0.0] * len(users)
users

,user.id,user.screen_name,user.location,user.verified,user.followers_count,user.friends_count,user.statuses_count,user.created_at,klout_score
0,771830969934438401,SPNGIRL420,"New Port Richey, FL",False,21,95,242,2016-09-02 22:03:38,0.0
1,757948847184240646,techno_nomadic,"Bushnell, FL",False,35,112,1061,2016-07-26 14:41:01,0.0
2,24633648,TeamChais,"Tampa, Florida",False,3714,3712,18395,2009-03-16 02:23:05,0.0
3,815777993733898240,thescarybeary,Earth,False,36,172,1207,2017-01-02 04:33:24,0.0
4,71023584,WFLAPaulM,"Tampa Bay, FL",True,2992,457,28814,2009-09-02 17:36:44,0.0
5,2284127615,cabaristas,"Port Charlotte, FL",False,1441,150,3293,2014-01-15 16:34:34,0.0
6,2876810176,JoannGuimares,"Tampa, FL",False,24,49,561,2014-11-14 18:35:50,0.0
7,4833325693,alexis_renee18,California Florida,False,206,189,4990,2016-01-30 03:54:30,0.0
8,773983063936294913,geribaby1947,"North Sarasota, FL",False,1190,1684,13374,2016-09-08 20:35:17,0.0
10,123717228,_OcalaFL,"Ocala, FL",False,294,4,31591,2010-03-17 01:04:59,0.0


In [18]:
places = mydata[['location.name','location.displayName','location.twitter_country_code','location.geo.coordinates']]
places = places.drop_duplicates(subset = ['location.name'])
# places['location.displayName'] = places['location.displayName'].apply(normalizeTweetText)
places.reset_index(drop = True)
places

,location.name,location.displayName,location.twitter_country_code,location.geo.coordinates
0,Bayonet Point,"Bayonet Point, FL",US,"[[[-82.733975, 28.299909], [-82.733975, 28.350..."
1,Clearwater,"Clearwater, FL",US,"[[[-82.831674, 27.935178], [-82.831674, 28.050..."
2,Florida,"Florida, USA",US,"[[[-87.634643, 24.396308], [-87.634643, 31.001..."
3,Ridge Wood Heights,"Ridge Wood Heights, FL",US,"[[[-82.523307, 27.276661], [-82.523307, 27.298..."
4,"The NewsCenter TBO.com, Tampa Tribune, News C...","The NewsCenter (TBO.com, Tampa Tribune, News C...",US,"[[[-82.460589, 27.944825], [-82.460589, 27.944..."
5,Charlotte Harbor,"Charlotte Harbor, FL",US,"[[[-82.084626, 26.946308], [-82.084626, 26.975..."
6,Temple Terrace,"Temple Terrace, FL",US,"[[[-82.40189, 28.011287], [-82.40189, 28.07369..."
7,East Lake-Orient Park,"East Lake-Orient Park, FL",US,"[[[-82.410125, 27.965019], [-82.410125, 28.031..."
8,Tallevast,"Tallevast, FL",US,"[[[-82.563644, 27.388731], [-82.563644, 27.429..."
10,Ocala,"Ocala, FL",US,"[[[-82.293902, 28.962902], [-82.293902, 29.277..."
